In [1]:
from unityagents import UnityEnvironment
import numpy as np

import matplotlib.pyplot as plt
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ", device)

env = UnityEnvironment(file_name="../Tennis_Linux/Tennis.x86_64", no_graphics=False)

brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]

using device:  cuda:0


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [3]:
# print("training the DDPG model")

# agent1 = Agent(state_size=state_size * 2, action_size=action_size)
# agent2 = Agent(state_size=state_size * 2, action_size=action_size)

# agent1.online_actor.load_state_dict(torch.load('actor_model1.path'))
# agent1.online_critic.load_state_dict(torch.load('critic_model1.path'))

# agent2.online_actor.load_state_dict(torch.load('actor_model2.path'))
# agent2.online_critic.load_state_dict(torch.load('critic_model2.path'))

In [4]:
# num_episodes = 10

# agent_scores = np.zeros(num_agents)

# for episode in range(1, num_episodes+1):

#     env_info = env.reset(train_mode=False)[brain_name]

#     state = env_info.vector_observations
#     state = np.reshape(state,(1,48))

#     agent1.reset()
#     agent2.reset()



#     while True:

#         action1 = agent1.act(state, add_noise=False)
#         action2 = agent2.act(state, add_noise=False)

#         action =[action1, action2]            
#         env_info = env.step(action)[brain_name]
#         next_state = env_info.vector_observations
#         next_state = np.reshape(next_state, (1, -1))
#         rewards = env_info.rewards
#         dones = env_info.local_done

#         agent_scores += rewards

#         state = next_state

#         if np.any(dones):
#             break


#     print("\r Agent 1: {} x {} : Agent 2".format(agent_scores[0], agent_scores[1]), end="", flush=True)

# print("Agent {} wins with {} points".format(np.argmax(agent_scores), np.max(agent_scores)))

In [5]:
from maddpg_agents import Maddpg
from hyperparameters import *
import torch
from collections import namedtuple, deque
import random
import matplotlib.pyplot as plt
import pandas as pd


In [6]:

# # Seeding
# np.random.seed(SEED)
# torch.manual_seed(SEED)

# # Instantiate the MADDPG agents
# maddpg = Maddpg(state_size, action_size, num_agents, SEED)

# maddpg.agents[0].critic_local.load_state_dict(torch.load('checkpoint_critic_local_0.pth'))
# maddpg.agents[0].critic_target.load_state_dict(torch.load('checkpoint_critic_target_0.pth'))
# maddpg.agents[0].actor_local.load_state_dict(torch.load('checkpoint_actor_local_0.pth'))
# maddpg.agents[0].actor_target.load_state_dict(torch.load('checkpoint_actor_target_0.pth'))



# maddpg.agents[1].critic_local.load_state_dict(torch.load('checkpoint_critic_local_1.pth'))
# maddpg.agents[1].critic_target.load_state_dict(torch.load('checkpoint_critic_target_1.pth'))
# maddpg.agents[1].actor_local.load_state_dict(torch.load('checkpoint_actor_local_1.pth'))
# maddpg.agents[1].actor_target.load_state_dict(torch.load('checkpoint_actor_target_1.pth'))


In [7]:
# all_steps=0
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     noise = NOISE
#     while True:
#         all_steps+=1
        
#         actions = maddpg.act(states, noise)                    # retrieve actions to performe for each agents 
#         noise *= NOISE_REDUCTION                               # Decrease action noise
#         env_info = env.step(actions)[brain_name]               # send all actions to tne environment
#         next_states = env_info.vector_observations             # get next state for each agent
#         rewards = env_info.rewards                             # get reward (for each agent)
#         dones = env_info.local_done                            # see if episode finished

#         # Save experience in replay memory, and use random sample from buffer to learn
#         maddpg.step(states, actions, rewards, next_states, dones, all_steps)  
#         scores += env_info.rewards                             # update the score (for each agent)
#         states = next_states                                   # roll over states to next time step
#         if np.any(dones):                                      # exit loop if episode finished
#             #print("   ** Debug: episode= {} steps={} rewards={} dones={}".format(i_episode, steps,rewards,dones))
#             break

#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

In [8]:

# Seeding
np.random.seed(SEED)
torch.manual_seed(SEED)

# Instantiate the MADDPG agents
maddpg = Maddpg(state_size, action_size, num_agents, SEED)

# maddpg.agents[0].critic_local.load_state_dict(torch.load('checkpoint_critic_local_0.pth'))
# maddpg.agents[0].critic_target.load_state_dict(torch.load('checkpoint_critic_target_0.pth'))
maddpg.agents[0].actor_local.load_state_dict(torch.load('checkpoint_actor_local_0.pth'))
# maddpg.agents[0].actor_target.load_state_dict(torch.load('checkpoint_actor_target_0.pth'))



# maddpg.agents[1].critic_local.load_state_dict(torch.load('checkpoint_critic_local_1.pth'))
# maddpg.agents[1].critic_target.load_state_dict(torch.load('checkpoint_critic_target_1.pth'))
maddpg.agents[1].actor_local.load_state_dict(torch.load('checkpoint_actor_local_1.pth'))
# maddpg.agents[1].actor_target.load_state_dict(torch.load('checkpoint_actor_target_1.pth'))


all_steps=0
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    noise = NOISE
    while True:
        all_steps+=1
        
        actions = maddpg.act(states, noise)                    # retrieve actions to performe for each agents 
        noise *= NOISE_REDUCTION                               # Decrease action noise
        env_info = env.step(actions)[brain_name]               # send all actions to tne environment
        next_states = env_info.vector_observations             # get next state for each agent
        rewards = env_info.rewards                             # get reward (for each agent)
        dones = env_info.local_done                            # see if episode finished

        # Save experience in replay memory, and use random sample from buffer to learn
        maddpg.step(states, actions, rewards, next_states, dones, all_steps)  
        scores += env_info.rewards                             # update the score (for each agent)
        states = next_states                                   # roll over states to next time step
        if np.any(dones):                                      # exit loop if episode finished
            #print("   ** Debug: episode= {} steps={} rewards={} dones={}".format(i_episode, steps,rewards,dones))
            break

    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

/home/pongsasit/anaconda3/envs/drlnd/lib/python3.6/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/media/pongsasit/480/deepRlUdacity/deep-reinforcement-learning/p3_collab-compet/tennis2/maddpg_agents.py:131: UserWarning: Using a target size (torch.Size([200, 2])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(Q_expected, Q_targets)


Score (max over agents) from episode 1: 0.6900000106543303
Score (max over agents) from episode 2: 0.10000000149011612
Score (max over agents) from episode 3: 0.10000000149011612
Score (max over agents) from episode 4: 0.09000000171363354
Score (max over agents) from episode 5: 0.09000000171363354


In [9]:
env.close()

In [10]:
# # agent1.online_actor.load_state_dict(torch.load('actor_model1.path'))

# # Monitor the score    
# scores_deque = deque(maxlen=100)
# all_scores = []
# all_avg_score = []


# # Intialize amplitude OU noise (will decay during training)
# noise = NOISE

# all_steps =0   # Monitor total number of steps performed

# # Training Loop
# for i_episode in range(21):  

#     env_info = env.reset(train_mode=False)[brain_name]          # reset the environment  
# #     maddpg.reset()                                             # reset the agents 

#     states = env_info.vector_observations                      # get the current state for each agent 
#     scores = np.zeros(num_agents)                              # initialize the score (for each agent)

#     for steps in range(NB_STEPS):

#         all_steps+=1

#         actions = maddpg.act(states, noise)                    # retrieve actions to performe for each agents 
#         noise *= NOISE_REDUCTION                               # Decrease action noise
#         env_info = env.step(actions)[brain_name]               # send all actions to tne environment
#         next_states = env_info.vector_observations             # get next state for each agent
#         rewards = env_info.rewards                             # get reward (for each agent)
#         dones = env_info.local_done                            # see if episode finished

#         # Save experience in replay memory, and use random sample from buffer to learn
#         maddpg.step(states, actions, rewards, next_states, dones, i_episode)  

#         scores += env_info.rewards                             # update the score (for each agent)
#         states = next_states                                   # roll over states to next time step
#         if np.any(dones):                                      # exit loop if episode finished
#             #print("   ** Debug: episode= {} steps={} rewards={} dones={}".format(i_episode, steps,rewards,dones))
#             break

# print("\r Agent 1: {} x {} : Agent 2".format(scores[0], scores[1]), end="", flush=True)
# print("Agent {} wins with {} points".format(np.argmax(scores), np.max(scores)))

In [11]:
env.close()

UnityEnvironmentException: No Unity environment is loaded.